# Setup

In [1]:
import os
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import random
import string
import glob
from PIL import Image
from matplotlib import cm
from scipy.stats import chisquare
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
from openpyxl import load_workbook
import itertools
import geopandas as gpd
import sklearn.neighbors
from unidecode import unidecode

# own functions 
from emelab_tools import *

warnings.filterwarnings('ignore')

# Paths
path = r'C:\Users\JoaquinFarina\OneDrive - Teachers College, Columbia University\Desktop\EMELAB\SIMCE'
EE_path = r'C:\Users\JoaquinFarina\OneDrive - Teachers College, Columbia University\Desktop\TESIS\MINEDUC'
path_cned = r'C:\Users\JoaquinFarina\OneDrive - Teachers College, Columbia University\Desktop\EMELAB\SIMCE\CNED\Cuadros_resumen'
path_esup = r'C:\Users\JoaquinFarina\OneDrive - Teachers College, Columbia University\Desktop\TESIS\MINEDUC\ESUP'

# years and grade levels per cohort
dict_agno_curso = {
    1: [[2005,2009,2012], [4,8,11]],
    2: [[2007, 2011, 2013], [4, 8, 10]],
    3: [[2008, 2014], [4, 10]],
    4: [[2009, 2013, 2015], [4, 8, 10]],
    5: [[2010, 2014, 2016], [4, 8, 10]],
    6: [[2011, 2013, 2015, 2017], [4, 6, 8, 10]],
    7: [[2012, 2014, 2018], [4, 6, 10]],
    8: [[2013, 2015, 2017], [4, 6, 8]],
    9: [[2012, 2014, 2016], [2, 4, 6]],
    10:[[2013, 2015, 2019], [2, 4, 8]],
    11: [[2014, 2016, 2018], [2, 4, 6, 10]]
}

# Apariciones por cohorte
n_app = [3, 3, 2, 3, 3, 4, 3, 3, 3, 3, 3]

# year cohorts in grade 2
agno_in_2b = [
    2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 1998, 
    2004, 2015, 2000, 1994, 2001, 2002, 2016
]

# years cohorts in grade 1 (truncated)
agno_in_1b = [x -1 for x in agno_in_2b]

# years cohorts in grade 4 
agno_in_4b = [x + 2 for x in agno_in_2b]

# years cohorts in grade (full)
cohort_1b = [2002,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013]


pendientes 
- Normalizar puntajes simce
- APC

# Questionnaire to dictionaries (parents, students, teachers)

## All questions to one large dictionary

In [134]:
dict_year_grade_type = {}
# walk subfolders
for root, dirs, files in os.walk(path):
    # walk all files in subfolders
     for file in files:
            # work with SEL directories 
            if  ('GLOSA' in file) & ('SEL' not in file):
                print(file)
                year = file[7:11]
                grade = file[5:7]
                path_file = os.path.join(root, file)
                if '~' not in path_file:
                    dict_simce = pd.read_excel(path_file,sheet_name=None,header=8)
                    for key in dict_simce.keys():
                        # keep questinaires
                        if (('_c' in key) & (key != '_comuna'))|('alu' in key):
                            val = year+ '_' + grade + key
                            if ((val == '2017_4b_cest_fa') | (val == '2017_4b_cest_fa')) & ('~' not in path_file) :
                                dict_simce = pd.read_excel(path_file,sheet_name=None,header=9)
                            data_simce = dict_simce[key]
                            # rectify colnames
                            for col in dict_simce[key].columns:
                                if 'Desc' in str(col) and '.' not in col:
                                    data_simce.rename(columns={col:'prompt'}, inplace=True)
                                if ('ombr' in str(col))|('Variable' == str(col)):
                                    data_simce.rename(columns={col:'key'}, inplace = True)
                                    
                            # fill in dict with data 
                            if 'cpad' in key:
                                dict_year_grade_type[year+ '_' + grade + key] = data_simce[['key','prompt']]
                            if 'cest' in key:
                                dict_year_grade_type[year+ '_' + grade + key] = data_simce[['key','prompt']]
                            if 'cprof' in key:
                                dict_year_grade_type[year+ '_' + grade + key] = data_simce[['key','prompt']]
                            if 'alu' in key:
                                dict_year_grade_type[year+ '_' + grade + '_alu'] = data_simce[['key','prompt']]

simce4b2002_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
simce2m2003_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
_cprof_mate
simce8b2004_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
_cprof_mate
_cprof_soc
_cprof_nat
simce4b2005_GLOSAS_PRIVADA-SEG.xlsx
_cprof
simce2m2006_GLOSAS_PRIVADA-SEG.xlsx
simce4b2006_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
simce4b2007_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
simce8b2007_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
_cprof_mate
_cprof_soc
_cprof_nat
simce2m2008_GLOSAS_PRIVADAS-SEG.xlsx
_cprof_lect
_cprof_mate
simce4b2008_GLOSAS_PRIVADAS-SEG.xlsx
_cprof_lect
simce4b2009_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
~$simce4b2009_GLOSAS_PRIVADA-SEG.xlsx
simce8b2009_GLOSAS_PRIVADA-SEG.xlsx
_cprof_lect
_cprof_mate
_cprof_soc
_cprof_nat
simce2m2010_GLOSAS-PRIVADA-SEG.xlsx
_cprof_lect
_cprof_mate
simce4b2010_GLOSAS-PRIVADA-SEG.xlsx
_cprof_lect
~$simce4b2010_GLOSAS-PRIVADA-SEG.xlsx
simce4b2011_GLOSAS-PRIVADA-SEG.xlsx
_cprof
simce8b2011_GLOSAS-PRIVADA-SEG.xlsx
_cprof_lect
_cprof_mate
_cprof_soc
_cprof_nat
~$si

## One dataframe for each type of questionnaire (parents, teachers, students)

In [3]:
# varname = key / label = prompt

In [172]:
cprof_dict

{'2002_4b_cprof_lect':               key                                             prompt
 0            agno                               Año de la evaluación
 1     codigo_bbdd                         Código único base de datos
 2      fecha_bbdd             Fecha en la que se construyó esta bbdd
 3             rbd              Rol base de datos del establecimiento
 4           anexo                  Anexo del rbd del establecimiento
 ..            ...                                                ...
 171  cprof_p36_07        Si pudiera, me cambiaría de establecimiento
 172     cprof_p37  Tomando en cuenta todos los aspectos de su tra...
 173     cprof_p38  Este año y en este establecimiento, ¿tuvo uste...
 174     cprof_p39  En años anteriores, cuando se reciben los resu...
 175     cprof_p40  A su juicio, ¿cuál es la principal razón para ...
 
 [176 rows x 2 columns],
 '2003_2m_cprof_lect':              key                                             prompt
 0           agno   

In [30]:
# dict for each type 
cpad_dict = {}
cprof_dict = {}
cest_dict = {}
alu_dict = {}

for key in dict_year_grade_type.keys():
    print(key)
    grade = key[5:7]
    if 'cpad' in key:
        cpad_dict[key] = dict_year_grade_type[key]
    if 'cpro' in key:
        cprof_dict[key] = dict_year_grade_type[key]
    if 'est' in key:
        cest_dict[key] = dict_year_grade_type[key]
    if 'alu' in key:
        alu_dict[key] = dict_year_grade_type[key]
        
# parents
df_cpad = pd.DataFrame()
for k in cpad_dict.keys():
    key_list = list(cpad_dict[k].key)
    prompt_list = list(cpad_dict[k].prompt)
    df_cpad[k+ '_key'] = key_list + (200-len(key_list))*[np.nan]
    df_cpad[k+ '_prompt'] = prompt_list + (200-len(prompt_list))*[np.nan]
    
# teachers
df_cprof = pd.DataFrame()
for k in cprof_dict.keys():
    key_list = list(cprof_dict[k].key)
    prompt_list = list(cprof_dict[k].prompt)
    df_cprof[k+ '_key'] = key_list + (240-len(key_list))*[np.nan]
    df_cprof[k+ '_prompt'] = prompt_list + (240-len(prompt_list))*[np.nan]
    
# students
df_cest = pd.DataFrame()
for k in cest_dict.keys():
    key_list = list(cest_dict[k].key)
    prompt_list = list(cest_dict[k].prompt)
    df_cest[k+ '_key'] = key_list + (240-len(key_list))*[np.nan]
    df_cest[k+ '_prompt'] = prompt_list + (240-len(prompt_list))*[np.nan]
    
# scores
df_alu = pd.DataFrame()
for k in alu_dict.keys():
    key_list = list(alu_dict[k].key)
    prompt_list = list(alu_dict[k].prompt)
    df_alu[k + '_key'] = key_list + (240-len(key_list))*[np.nan]
    df_alu[k + '_prompt'] = prompt_list + (240-len(prompt_list))*[np.nan]

2002_4b_alu
2002_4b_cpad
2002_4b_cprof_lect
2003_2m_alu
2003_2m_cpad
2003_2m_cprof_lect
2003_2m_cprof_mate
2004_8b_alu
2004_8b_cpad
2004_8b_cprof_lect
2004_8b_cprof_mate
2004_8b_cprof_soc
2004_8b_cprof_nat
2005_4b_alu
2005_4b_cpad
2005_4b_cprof
2006_2m_alu
2006_2m_cpad
2006_4b_alu
2006_4b_cpad
2006_4b_cprof_lect
2007_4b_alu
2007_4b_cpad
2007_4b_cprof_lect
2007_8b_alu
2007_8b_cpad
2007_8b_cprof_lect
2007_8b_cprof_mate
2007_8b_cprof_soc
2007_8b_cprof_nat
2008_2m_alu
2008_2m_cpad
2008_2m_cprof_lect
2008_2m_cprof_mate
2008_4b_alu
2008_4b_cpad
2008_4b_cprof_lect
2009_4b_alu
2009_4b_cpad
2009_4b_cprof_lect
2009_4b_cest
2009_8b_alu
2009_8b_cpad
2009_8b_cprof_lect
2009_8b_cprof_mate
2009_8b_cprof_soc
2009_8b_cprof_nat
2009_8b_cest
2010_2m_alu
2010_2m_cpad
2010_2m_cprof_lect
2010_2m_cprof_mate
2010_2m_cest
2010_4b_alu
2010_4b_cpad
2010_4b_cprof_lect
2010_4b_cest
2011_4b_alu
2011_4b_cpad
2011_4b_cprof
2011_4b_cest
2011_8b_alu
2011_8b_cpad
2011_8b_cprof_lect
2011_8b_cprof_mate
2011_8b_cprof_soc
2

# Match questions across questionnaires

Search within questionnaires ant matching across.

## Parents

In [31]:
# Household income
income_question = 'En un mes normal, ¿en cuál de los siguientes rangos se encuentra la suma de los ingresos de todas las personas que aportan al hogar donde vive el estudiante?'
match_df_income = get_questions_from_questionnaire(income_question,df_cpad,'cpad')

# Parental expectations
parental_expectations_question = '¿Cuál cree usted que es el nivel de educación más alto que el estudiante podrá completar en el futuro?'
match_df_parental_exp = get_questions_from_questionnaire(parental_expectations_question,df_cpad,'cpad')

# Age of respondent
age_question = '¿Cuál es su edad?'
match_df_age = get_questions_from_questionnaire(age_question,df_cpad,'cpad')

# Relation to student of respondent 
relation_to_student_question = '¿Cuál es su relación de parentesco con el estudiante?'
match_df_relation = get_questions_from_questionnaire(relation_to_student_question,df_cpad,'cpad')

# Mom educational level
mom_education_question = '¿Hasta qué nivel educacional llegó la madre (o madrastra) del estudiante?'
match_df_mom_ed = get_questions_from_questionnaire(mom_education_question,df_cpad,'cpad')

# Dad educational level
dad_education_question = '¿Hasta qué nivel educacional llegó el padre (o padrastro) del estudiante?'
match_df_dad_ed = get_questions_from_questionnaire(dad_education_question,df_cpad,'cpad')

# Parental aspirations
aspirations_question = '¿A qué nivel educacional le gustaría a ud. que llegara el estudiante?'
match_df_aspirations = get_questions_from_questionnaire(aspirations_question,df_cpad,'cpad')

# Reasons for parental expectations 
reasons_question = '¿cuál es la principal razón por la que el alumno lograría llegar a ese nivel educacional?'
match_df_reasons = get_questions_from_questionnaire(reasons_question,df_cpad,'cpad')

# School support 
school_support_question = 'El apoyo que el establecimiento entrega a los padres para que eduquen a sus hijos (por ejemplo, los orientan en cómo ayudar a sus hijos en las tareas escolares, los orientan en la crianza, se realizan talleres para padres, etc.).'
match_df_school_support = get_questions_from_questionnaire(school_support_question,df_cpad,'cpad')


 2002_4b_cpad_prompt
('En promedio, el ingreso económico mensual total del hogar donde vive el alumno está entre:', 51)

 2003_2m_cpad_prompt
('En general, ¿con qué frecuencia realiza usted las siguientes actividades con el alumno?\n\nConversar sobre noticias y temas de actualidad política o social', 49)

 2004_8b_cpad_prompt
('En un mes normal, el ingreso económico total del hogar donde vive el alumno está:', 55)

 2005_4b_cpad_prompt
('En un mes normal, ¿cuáles de los siguientes gastos relacionados con la educación del alumno realiza usted?\n\nMateriales escolares', 58)

 2006_2m_cpad_prompt
('En un mes normal, la suma de los ingresos de todas las personas que viven en el hogar del alumno está:', 67)

 2006_4b_cpad_prompt
('En un mes normal, la suma de los ingresos de todas las personas que viven en el hogar del alumno está:', 67)

 2007_4b_cpad_prompt
('En un mes normal, la suma de los ingresos de todas las personas que viven en el hogar del alumno está:', 67)

 2007_8b_cpad_prompt

In [32]:
parents_match_dict = {
    'income':match_df_income,
    'expectations':match_df_parental_exp,
    'age':match_df_age,
    'relation':match_df_relation,
    'mom_ed':match_df_mom_ed,
    'dad_ed': match_df_dad_ed,
    'aspirations': match_df_aspirations,
    'reasons': match_df_reasons,
    'support': match_df_school_support
}

## Teachers

In [142]:
def get_questions_from_questionnaire_prof(question,df_questionaire,case):
    from fuzzywuzzy import process
    from fuzzywuzzy import fuzz
    import pandas as pd

    match_list = []
    for x in [x for x in df_questionaire.columns if 'prompt' in x]:
        print('\n',x)
        vals= list(df_questionaire[x].values)
        vals = [str(x) for x in vals if str(x) != 'nan']
        match = process.extractOne(question, vals, scorer=fuzz.ratio)     
        print(match)
        varname =  df_questionaire[df_questionaire[x] == match[0]][x.replace('prompt','key')].values[0]

        curso =  x[5:7]
        agno = x[:4]
        subject = x.split('_')[3]

        match_list = match_list + [[curso+agno+'_'+case + '_' + subject,varname]]

    match_df =  pd.DataFrame(match_list,columns = ['grade_year','key'])
    return match_df

In [272]:
age_question_t = "¿Cuál es su edad?"
match_df_age_question_t = get_questions_from_questionnaire_prof(age_question_t,df_cprof,'cprof')

experience_school_question = "¿Cuántos años de experiencia tiene usted como docente en este colegio?"
match_df_age_experience_school_question = get_questions_from_questionnaire_prof(experience_school_question,df_cprof,'cprof')

experience_total_question = "¿Cuántos años de experiencia en total tiene usted como docente?"
match_df_age_experience_total_question = get_questions_from_questionnaire_prof(experience_total_question,df_cprof,'cprof')

grade_level_question = "Grado en el que se rinde la evaluación"
match_df_grade_level_question = get_questions_from_questionnaire_prof(grade_level_question,df_cprof,'cprof')

class_code_question = "Código de curso (para vincular con cuestionario docente)"
match_df_class_code_question = get_questions_from_questionnaire_prof(class_code_question,df_cprof,'cprof')

class_letter_question = "Letra del curso"
match_df_class_letter_question = get_questions_from_questionnaire_prof(class_letter_question,df_cprof,'cprof')

teacher_expectations_question = "Pensando en el futuro, ¿cuál cree usted que es el nivel educacional más alto que completará la mayoría de los estudiantes de este curso?"
match_df_teacher_expectations = get_questions_from_questionnaire_prof(teacher_expectations_question,df_cprof,'cprof')



 2002_4b_cprof_lect_prompt
('Buscar información y comentar acerca de las características más relevantes del planeta tierra, la galaxia y los componentes del sistema solar', 48)

 2003_2m_cprof_lect_prompt
('¿Con qué frecuencia realiza usted las siguientes actividades en sus clases con los alumnos de este curso?\n\nUsa estrategias de trabajo grupal', 47)

 2003_2m_cprof_mate_prompt
('¿Con qué frecuencia realiza usted las siguientes actividades en sus clases con los alumnos de este curso?\n\nUsa estrategias de trabajo grupal', 47)

 2004_8b_cprof_lect_prompt
('En su opinión, ¿cuál de estos niveles de estudio cree usted que alcanzará la mayoría de sus alumnos de 8º básico?', 55)

 2004_8b_cprof_mate_prompt
('En su opinión, ¿cuál de estos niveles de estudio cree usted que alcanzará la mayoría de sus alumnos de 8º básico?', 55)

 2004_8b_cprof_soc_prompt
('En su opinión, ¿cuál de estos niveles de estudio cree usted que alcanzará la mayoría de sus alumnos de 8º básico?', 55)

 2004_8b_cprof

In [305]:
# 1 = No estoy seguro de que puedan terminar la enseñanza básica
# 2 = Creo que terminarán la enseñanza básica
# 3 = No estoy seguro de que puedan terminar la enseñanza media
# 4 = Creo que terminarán la enseñanza media
# 5 = Creo que podrán estudiar una carrera en un Instituto Profesional o Centro de Formación Técnica
# 6 = Creo que podrán estudiar una carrera en una universidad

match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '4b2002_cprof_lect', 'key'] = 'cprof_p32'

# 2003 no question
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '2m2003_cprof_lect', 'key'] = np.nan
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '2m2003_cprof_mate', 'key'] = np.nan

# 2012 3m no question 
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '3m2012_cprof_prompt', 'key'] = np.nan

# 2015 poco 
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '4b2015_cprof_prompt', 'key'] = np.nan
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '4b2015_cprof_prompt', 'grade_year'] = '4b2015_cprof_lect'

match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '6b2015_cprof_lect', 'key'] = np.nan
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '6b2015_cprof_mate', 'key'] = np.nan

# 2022 raro 
# 2m lect 
#¿Cuán difícil es para usted realizar las siguientes acciones utilizando un dispositivo digital (por ejemplo, computador, celular, tablet, etc.)?	"1: La mayoría no terminará la enseñanza media
#2: La mayoría completará la enseñanza media técnico profesional
#3: La mayoría completará la enseñanza media científico humanista
#4: La mayoría completará una carrera en un instituto profesional o en un centro de formación técnica
#5: La mayoría completará una carrera en una universidad
#6: La mayoría completará un programa de postgrado
#0: Vacio
#99: doble marca"
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '2m2022_cprof_lect', 'key'] = 'cprof_p18_07'
# mat nada 
match_df_teacher_expectations.loc[match_df_teacher_expectations.grade_year == '2m2022_cprof_mate', 'key'] = np.nan


# 4b 2007 
#1: No terminará la Enseñanza Básica
#2: Completará la Enseñanza Básica
#3: No terminará la Enseñanza Media
#4: Completará la Enseñanza Media en un TP
#5: Completará la Enseñanza Media en un CH
#6: Completará una carrera en un IP o un CFT
#7: Completará una carrera en una U
#8: Completará un postgrado en la U
#9: Doble marca
#0: Vacío


In [127]:
teachers_match_dict = {
    'age':match_df_age_question_t,
    'experience_in_school':match_df_age_experience_school_question,
    'experience_total':match_df_age_experience_total_question,
    'grade_level':match_df_grade_level_question,
    'class_code':match_df_class_code_question,
    'class_letter': match_df_class_letter_question,
    'teacher_expectations': match_df_teacher_expectations
}

## Scores

In [69]:
match_df_simce_mat = get_questions_from_questionnaire('puntaje estudiante simce matemática',df_alu,'alu') # ojo que 2b nunca rinde matematicas
match_df_simce_lect = get_questions_from_questionnaire('puntaje estudiante simce lectura',df_alu,'alu')

alu_match_dict = {
    'math': match_df_simce_mat,
    'reading': match_df_simce_lect
}


 2002_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2003_2m_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2004_8b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2005_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2006_2m_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2006_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2007_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2007_8b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2008_2m_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2008_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2009_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2009_8b_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2010_2m_alu_prompt
('Puntaje del estudiante en SIMCE Matemática', 91)

 2010_4b_alu_prompt
('Puntaje del estudiante en SIMCE Matemátic

## Students

In [105]:
student_expectations = get_questions_from_questionnaire('Pensando en el futuro, ¿cuál es el nive de educación más alto que crees que vas a poder completar?',df_cest,'cest') 


 2009_4b_cest_prompt
('Pensando en tu experiencia en tu escuela, ¿estás de acuerdo con las siguientes frases?\n\nSiento que soy tan capaz de aprender como el resto de mis compañeros y\ncompañeras de curso', 48)

 2009_8b_cest_prompt
('Pensando en el futuro, ¿cuál es el nivel de educación más alto que crees que vas a poder completar?', 99)

 2010_2m_cest_prompt
('Pensando en el futuro, ¿cuál es el nivel de educación más alto que crees que vas a poder completar?', 99)

 2010_4b_cest_prompt
('Pensando en cómo te sientes actualmente, ¿qué tan contento vienes a tu escuela?', 48)

 2011_4b_cest_prompt
('En general, entiendo muy poco de lo que me pasan en clases.', 49)

 2011_8b_cest_prompt
('Pensando en el futuro, ¿cuál es el nive de educación más alto que crees que vas a poder completar?', 100)

 2012_2m_cest_prompt
('Pensando en tu futuro, ¿cuál es el nivel de educación más alto que crees que vas a poder completar? ', 97)

 2012_3m_cest_prompt
('¿Cuántas veces a la semana has asistido a c

In [114]:
# los que si son 
est_asp_real = ['8b2009_cest','2m2010_cest','8b2011_cest','2m2012_cest','2m2013_cest','8b2013_cest','2m2014_cest',
                '8b2014_cest','2m2015_cest','8b2015_cest','2m2016_cest','2m2016_cest','2m2017_cest','2m2017_cest',
                '8b2017_cest','8b2017_cest','2m2018_cest','8b2019_cest']
student_expectations = student_expectations[student_expectations.grade_year.isin(est_asp_real)]

In [106]:
student_expectations

,grade_year,key
0,4b2009_cest,cest_p04_01
1,8b2009_cest,cest_p23
2,2m2010_cest,cest_p23
3,4b2010_cest,cest_p03
4,4b2011_cest,cest_p06_03
5,8b2011_cest,cest_p05
6,2m2012_cest,cest_p09
7,3m2012_cest,cest_p08
8,4b2012_cest,cest_p24
9,2m2013_cest,cest_p06


# Process actual data

## Parents

In [ ]:
#old
#match_dict = {
#1:match_df_income,
#2:match_df_parental_exp,
#3:match_df_age,
#4:match_df_relation,
#5:match_df_mom_ed}

In [36]:
def add_parental_expectations(data,file, parents_match_dict):
    
    match_df = parents_match_dict['expectations']
    question = match_df[match_df.grade_year == file[5:16]].key.values[0]
    
    q_list = []
    if ('_0' in question) & ((t == '2016_6b_cpad')| (t=='2018_6b_cpad')):
        question = question[:-2]  
        
    q_list = q_list + [question]
    
    
    if '4b2002' in file:
        q_list = ['cpad_p20']
    
    if '4b2007' in file:
        q_list = ['cpad_p16']
        
    if '4b2005' in file:
        q_list = ['cpad_p20']
        
    if '4b2006' in file:
        q_list = ['cpad_p22']       
        
    if '2m2006' in file:
        q_list = ['cpad_p23']
        
    if '8b2007' in file:
        q_list = ['cpad_p14']
            
    # parental exp 
    if ('simce4b2008' in file):
        data['count'] = data['cpad_p15_01'] 
        for i in range(2,9):                    
            data['cpad_p15_01'] = data['cpad_p15_01'] + i*data['cpad_p15_0' + str(i)]
            data['count'] = data['count'] + data['cpad_p15_0' + str(i)]
    
        data['parental_exp'] = data.cpad_p15_01*(data['count'] == 1)
        q_list = ['parental_exp'] + q_list[1:]
    
    if ('simce2m2008' in file):
        data['count'] = data['cpad_p16_01'] 
        for i in range(2,7):                    
            data['cpad_p16_01'] = data['cpad_p16_01'] + i*data['cpad_p16_0' + str(i)]
            data['count'] = data['count'] + data['cpad_p16_0' + str(i)]
    
        data['parental_exp'] = data.cpad_p16_01*(data['count'] == 1)
        q_list = ['parental_exp'] + q_list[1:]
        
        
    # parental exp
    if '2003' in file:
        data.cpad_p20.replace(2, 4, inplace=True)
        data.cpad_p20.replace(3, 5, inplace=True)
        data.cpad_p20.replace(1, 3, inplace=True)
    
    if '2002' in file:
        data.cpad_p20.replace(2, 1, inplace=True)
        data.cpad_p20.replace(3, 1, inplace=True)
        data.cpad_p20.replace(4, 2, inplace=True)
        data.cpad_p20.replace(5, 3, inplace=True)
        data.cpad_p20.replace(6, 4, inplace=True)
        data.cpad_p20.replace(7, 5, inplace=True)
        data.cpad_p20.replace(8, 6, inplace=True)
    
    if '8b2004' in file:
        data.cpad_p11.replace(4, 5, inplace=True)
        data.cpad_p11.replace(3, 4, inplace=True)
        data.cpad_p11.replace(2, 3, inplace=True)
 
    if '4b2006' in file:
        data.cpad_p22.replace(2, 1, inplace=True)
        data.cpad_p22.replace(3, 1, inplace=True)
        data.cpad_p22.replace(4, 2, inplace=True)
        data.cpad_p22.replace(5, 3, inplace=True)
        data.cpad_p22.replace(6, 4, inplace=True)
        data.cpad_p22.replace(7, 5, inplace=True)
        data.cpad_p22.replace(8, 6, inplace=True)
        
    if '4b2005' in file:
        data.cpad_p20.replace(0, 1, inplace=True)
        data.cpad_p20.replace(2, 1, inplace=True)
        data.cpad_p20.replace(3, 2, inplace=True)
        data.cpad_p20.replace(4, 3, inplace=True)
        data.cpad_p20.replace(5, 4, inplace=True)
        data.cpad_p20.replace(6, 5, inplace=True)
        data.cpad_p20.replace(7, 6, inplace=True)
        
    if '4b2007' in file:
        data.cpad_p16.replace(2, 1, inplace=True)
        data.cpad_p16.replace(3, 1, inplace=True)
        data.cpad_p16.replace(4, 2, inplace=True)
        data.cpad_p16.replace(5, 3, inplace=True)
        data.cpad_p16.replace(6, 4, inplace=True)
        data.cpad_p16.replace(7, 5, inplace=True)
        data.cpad_p16.replace(8, 6, inplace=True)
        
    if '8b2007' in file:
        data.cpad_p14.replace(2, 1, inplace=True)
        data.cpad_p14.replace(3, 1, inplace=True)
        data.cpad_p14.replace(4, 2, inplace=True)
        data.cpad_p14.replace(5, 3, inplace=True)
        data.cpad_p14.replace(6, 4, inplace=True)
        data.cpad_p14.replace(7, 5, inplace=True)
        data.cpad_p14.replace(8, 6, inplace=True)
        data.cpad_p14.replace(9, np.nan, inplace=True)
        
    if '4b2008' in file:
        data.parental_exp.replace(2, 1, inplace=True)
        data.parental_exp.replace(3, 1, inplace=True)
        data.parental_exp.replace(4, 2, inplace=True)
        data.parental_exp.replace(5, 3, inplace=True)
        data.parental_exp.replace(6, 4, inplace=True)
        data.parental_exp.replace(7, 5, inplace=True)
        data.parental_exp.replace(8, 6, inplace=True)
        data.parental_exp.replace(9, np.nan, inplace=True)
    
    return data, q_list

In [37]:
def add_income(data,file, parents_match_dict, q_list):

    match_df = parents_match_dict['income']
    question = match_df[match_df.grade_year == file[5:16]].key.values[0]
    
    if ('_0' in question) & ((t == '2016_6b_cpad')| (t=='2018_6b_cpad')):
        question = question[:-2]   
        
    q_list = q_list + [question]
        
    if '2m2003' in file:
        q_list = q_list[:-1] + ['cpad_p05']   
        
    if '8b2007' in file:
        q_list = q_list[:-1] + ['cpad_p09']
    
    # income
    if ('simce4b2008' in file)|('simce2m2008' in file):
        data['income2'] = data['cpad_p07_01'] 
        for i in range(2,14):
            if i < 10:
                data['cpad_p07_0' + str(i)] = i*data['cpad_p07_0' + str(i)]
                data['income2'] = data[['income2','cpad_p07_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p07_' + str(i)] = i*data['cpad_p07_' + str(i)]
                data['income2'] = data[['income2','cpad_p07_' + str(i)]].max(axis = 1)

        data['income'] = data.income2
        q_list = q_list[:-1] + ['income'] 
        
    if ('simce4b2010' in file)|('simce2m2010' in file):
        data['income2'] = data['cpad_p11_01'] 
        for i in range(2,14):
            if i < 10:
                data['cpad_p11_0' + str(i)] = i*data['cpad_p11_0' + str(i)]
                data['income2'] = data[['income2','cpad_p11_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p11_' + str(i)] = i*data['cpad_p11_' + str(i)]
                data['income2'] = data[['income2','cpad_p11_' + str(i)]].max(axis = 1)
                                        
        data['income'] = data.income2
        q_list = q_list[:-1] + ['income'] 
        
    if ('simce4b2011' in file) | ('simce8b2011' in file):
        data['income2'] = data['cpad_p10_01'] 
        for i in range(2,16):                            
            if i < 10:
                data['cpad_p10_0' + str(i)] = i*data['cpad_p10_0' + str(i)]
                data['income2'] = data[['income2','cpad_p10_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p10_' + str(i)] = i*data['cpad_p10_' + str(i)]
                data['income2'] = data[['income2','cpad_p10_' + str(i)]].max(axis = 1)
                                        
        data['income'] = data.income2
        q_list = q_list[:-1] + ['income'] 

        
    if ('simce4b2009' in file)|('simce8b2009' in file):
        data['income2'] = data['cpad_p10_01'] 
        for i in range(2,16):                            
            if i < 10:
                data['cpad_p10_0' + str(i)] = i*data['cpad_p10_0' + str(i)]
                data['income2'] = data[['income2','cpad_p10_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p10_' + str(i)] = i*data['cpad_p10_' + str(i)]
                data['income2'] = data[['income2','cpad_p10_' + str(i)]].max(axis = 1)
                                        
        data['income'] = data.income2
        q_list = q_list[:-1] + ['income'] 
        
    if 'simce4b2005' in file:
        data['income'] = data.cpad_p12+1
        q_list = q_list[:-1] + ['income'] 
            
    data[q_list[-1]] = data[q_list[-1]].replace([0,16,17,18,19,20,21,22,32,99,100], np.nan)
    
    return data, q_list

In [38]:
dict_mom_ed_two_digit ={11.0:2, 12.0:3, 13.0:4, 14.0:5, 15.0:6, 16.0:7, 17.0:8, 18.0:9, 21.0:10, 22.0:11, 
                        23.0:12,24.0:13, 31.0:10, 32.0:11, 33.0:12, 34.0:14, 35.0:14, 41.0:15, 42.0:16, 
                        43.0:16, 44.0:16, 51.0:15,52.0:15, 53.0:16, 54.0:16, 55.0:16, 61.0:17, 62.0:17, 
                        63.0:17, 64.0:18, 65.0:18, 66.0:18, 67.0:18,71.0:18, 72.0:18, 81.0:19, 82.0:19, 
                        83.0:19, 84.0:19,85.0:19}

In [39]:
from collections import defaultdict

dict_mom_ed_doubled = defaultdict(lambda: np.nan)
dict_mom_ed_doubled_values ={'[1.0, 1.0]':2, '[1.0, 2.0]':3, '[1.0, 3.0]':4, '[1.0, 4.0]':5, '[1.0, 5.0]':6,'[1.0, 6.0]':7,'[1.0, 7.0]':8,'[1.0, 8.0]':9,
'[2.0, 1.0]':10,'[2.0, 2.0]':11,'[2.0, 3.0]':12,'[2.0, 4.0]':13,
'[3.0, 1.0]':10,'[3.0, 2.0]':11,'[3.0, 3.0]':12,'[3.0, 4.0]':14,'[3.0, 5.0]':14,
'[4.0, 1.0]':15,'[4.0, 2.0]':16,'[4.0, 3.0]':16,'[4.0, 4.0]':16,
'[5.0, 1.0]':15,'[5.0, 2.0]':15,'[5.0, 3.0]':16,'[5.0, 4.0]':16, '[5.0, 5.0]':16,
'[6.0, 1.0]':17,'[6.0, 2.0]':17,'[6.0, 3.0]':17,'[6.0, 4.0]':18, '[6.0, 5.0]':18,'[6.0, 6.0]':18,'[6.0, 7.0]':18,
'[7.0, 1.0]':18,'[7.0, 2.0]':18,
'[8.0, 1.0]':19,'[8.0, 2.0]':19,'[8.0, 3.0]':19,'[8.0, 4.0]':19, '[8.0, 5.0]':19,'[8.0, 6.0]':19}
                                             
for key, value in dict_mom_ed_doubled_values.items():
    dict_mom_ed_doubled[key] = value  

In [40]:
def correct_mom_ed_2004(row):
    val = str(list(row[['cpad_p13_01','cpad_p13_02']].values))
    return dict_mom_ed_doubled[val]

def correct_mom_ed_2002(row):
    val = str(list(row[['cpad_p09_01','cpad_p09_02']].values))
    return dict_mom_ed_doubled[val]

In [41]:
def correct_dad_ed_2004(row):
    val = str(list(row[['cpad_p15_01','cpad_p15_02']].values))
    return dict_mom_ed_doubled[val]

def correct_dad_ed_2002(row):
    val = str(list(row[['cpad_p08_01','cpad_p08_02']].values))
    return dict_mom_ed_doubled[val]

In [ ]:
1: No estudió
2: 1er año de educación básica
3: 2° año de educación básica
4: 3er año de educación básica
5: 4° año de educación básica
6: 5° año de educación básica
7: 6° año de educación básica
8: 7° año de educación básica
9: 8° año de educación básica
10: I año de educación media
11: II año de educación media
12: III año de educación media
13: IV año de educación media científico-humanista
14: IV o V año de educación media técnico-profesional o vocacional
15: Educación incompleta en un centro de formación técnica o instituto profesional
16: Educación completa en un centro de formación técnica o instituto profesional
17: Educación incompleta en una universidad
18: Educación completa en una universidad
19: Grado de magíster universitario
20: Grado de doctor universitario
21: No sabe o no recuerda
0: Vacío
99: doble marca
    
1: 9,2: 13,3: 14,4: 16,5: 16,6: 18,7: 18 ,8: 19


In [42]:
def add_mom_ed(data,file, parents_match_dict, q_list):

    match_df = parents_match_dict['mom_ed']
    question = match_df[match_df.grade_year == file[5:16]].key.values[0]
    
    if ('_0' in question) & ((t == '2016_6b_cpad')| (t=='2018_6b_cpad')):
        question = question[:-2]   
        
    q_list = q_list + [question]
    
    if '8b2007' in file:
        q_list = q_list[:-1] + ['cpad_p06']
        question = 'cpad_p06'
        
    if '8b2004' in file:
        q_list = q_list[:-1] + ['cpad_p13_01']   
        question = 'cpad_p13_01'
        
    if '2m2003' in file:
        q_list = q_list[:-1] + ['cpad_p06']
        question = 'cpad_p06'
    
    # mom ed 2008, 2009, 2010 y 2011 vienen separadas
    if ('simce4b2008' in file)|('simce2m2008' in file):
        data['count_ed'] = data['cpad_p06_01'] 
        for i in range(2,22):
            if i < 10:
                data['cpad_p06_0' + str(i)] =  i*data['cpad_p06_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p06_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p06_' + str(i)] = i*data['cpad_p06_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p06_' + str(i)]].max(axis = 1)
                              
        data['mom_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['mom_ed']
        question = 'mom_ed'
        # ajustar escala 2007 y 2008
        data[question].replace([0,1,99],np.nan,inplace=True)
        data[question] = data[question] -1 
        data[question] = data[question].replace(-1,np.nan)
    
    if "2006" in file: 
        data[question] = data[question] -1 
        data[question] = data[question].replace(-1,np.nan)
        
    if ('simce4b2009' in file)|('simce8b2009' in file):
        data['count_ed'] = data['cpad_p09_02_01'] 
        for i in range(2,22):
            if i < 10:
                data['cpad_p09_02_0' + str(i)] =  i*data['cpad_p09_02_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_02_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p09_02_' + str(i)] = i*data['cpad_p09_02_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_02_' + str(i)]].max(axis = 1)
                              
        data['mom_ed'] = data.count_ed           
        q_list = q_list[:-1] + ['mom_ed']
        
    # mom_ed 2010
    if ('simce4b2010' in file)|('simce2m2010' in file):
        data['count_ed'] = data['cpad_p10_01'] 
        for i in range(1,21):
            if i < 10:
                data['cpad_p10_0' + str(i)] =  i*data['cpad_p10_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p10_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p10_' + str(i)] = i*data['cpad_p10_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p10_' + str(i)]].max(axis = 1)
                              
        data['mom_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['mom_ed']
        
    if 'simce4b2011' in file:
        data['count_ed'] = data['cpad_p09_01'] 
        for i in range(1,21):                            
            if i < 10:
                data['cpad_p09_0' + str(i)] =  i*data['cpad_p09_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p09_' + str(i)] = i*data['cpad_p09_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_' + str(i)]].max(axis = 1)
                              
        data['mom_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['mom_ed']
                                
    if 'simce8b2011' in file:
        data['count_ed'] = data['cpad_p08_01'] 
        for i in range(1,21):                            
            if i < 10:
                data['cpad_p08_0' + str(i)] =  i*data['cpad_p08_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p08_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p08_' + str(i)] = i*data['cpad_p08_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p08_' + str(i)]].max(axis = 1)
                              
        data['mom_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['mom_ed']
       
    # mom_ed 2002, 2003 2004 y 2007 fix escala 
    if ('simce4b2007' in file)|('simce8b2007' in file):                       
        # ajustar escala 2007 y 2008
        data[question].replace([0,1,99],np.nan,inplace=True)
        data[question] = data[question] -1 
    
    

    if "2002" in file:                       
        data[question] = data.apply(correct_mom_ed_2002, axis=1)
        
    if 'simce8b2004' in file:                       
        data[question] = data.apply(correct_mom_ed_2004, axis=1)
        
    if ("2003" in file):                       
        data[question].replace(dict_mom_ed_two_digit,inplace=True)
     
    if "simce3m2012" in file:
        data.drop_duplicates(keep="first", inplace=True)
       
    if ('simce4b2005' in file):
        data['mom_ed'] = data.cpad_p08+1
        q_list = q_list[:-1] + ['mom_ed']
    
    
    data[q_list[-1]] = data[q_list[-1]].replace([0,98,99,100],np.nan)

    return data, q_list

In [43]:
def add_dad_ed(data,file, parents_match_dict, q_list):

    match_df = parents_match_dict['dad_ed']
    question = match_df[match_df.grade_year == file[5:16]].key.values[0]
    
    if ('_0' in question) & ((t == '2016_6b_cpad')| (t=='2018_6b_cpad')):
        question = question[:-2]   
        
    q_list = q_list + [question]
    
    if '8b2007' in file:
        q_list = q_list[:-1] + ['cpad_p05']
        question = 'cpad_p05'
        
    if '8b2004' in file:
        q_list = q_list[:-1] + ['cpad_p14_01']   
        question = 'cpad_p14_01'
        
    if '2m2003' in file:
        q_list = q_list[:-1] + ['cpad_p08']
        question = 'cpad_p08'
    
    # mom ed 2008, 2009, 2010 y 2011 vienen separadas
    if ('simce4b2008' in file)|('simce2m2008' in file):
        data['count_ed'] = data['cpad_p05_01'] 
        for i in range(2,22):
            if i < 10:
                data['cpad_p05_0' + str(i)] =  i*data['cpad_p05_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p05_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p05_' + str(i)] = i*data['cpad_p05_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p05_' + str(i)]].max(axis = 1)
                              
        data['dad_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['dad_ed']
        question = 'dad_ed'
        # ajustar escala 2007 y 2008
        data[question].replace([0,1,99],np.nan,inplace=True)
        data[question] = data[question] -1 
        data[question] = data[question].replace(-1,np.nan)
    
    if "2006" in file: 
        data[question] = data[question] -1 
        data[question] = data[question].replace(-1,np.nan)
        
    if ('simce4b2009' in file)|('simce8b2009' in file):
        data['count_ed'] = data['cpad_p09_01_01'] 
        for i in range(2,22):
            if i < 10:
                data['cpad_p09_01_0' + str(i)] =  i*data['cpad_p09_01_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_01_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p09_01_' + str(i)] = i*data['cpad_p09_01_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_01_' + str(i)]].max(axis = 1)
                              
        data['dad_ed'] = data.count_ed           
        q_list = q_list[:-1] + ['dad_ed']
        
    # mom_ed 2010
    if ('simce4b2010' in file)|('simce2m2010' in file):
        data['count_ed'] = data['cpad_p09_01'] 
        for i in range(1,21):
            if i < 10:
                data['cpad_p09_0' + str(i)] =  i*data['cpad_p09_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p09_' + str(i)] = i*data['cpad_p09_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_' + str(i)]].max(axis = 1)
                              
        data['dad_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['dad_ed']
        
    if 'simce4b2011' in file:
        data['count_ed'] = data['cpad_p08_01'] 
        for i in range(1,21):                            
            if i < 10:
                data['cpad_p08_0' + str(i)] =  i*data['cpad_p08_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p08_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p08_' + str(i)] = i*data['cpad_p08_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p08_' + str(i)]].max(axis = 1)
                              
        data['dad_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['dad_ed']
                                
    if 'simce8b2011' in file:
        data['count_ed'] = data['cpad_p09_01'] 
        for i in range(1,21):                            
            if i < 10:
                data['cpad_p09_0' + str(i)] =  i*data['cpad_p09_0' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_0' + str(i)]].max(axis = 1)
            else:
                data['cpad_p09_' + str(i)] = i*data['cpad_p09_' + str(i)]
                data['count_ed'] = data[['count_ed','cpad_p09_' + str(i)]].max(axis = 1)
                              
        data['dad_ed'] = data.count_ed   
        q_list = q_list[:-1] + ['dad_ed']
       
    # mom_ed 2002, 2003 2004 y 2007 fix escala 
    if ('simce4b2007' in file)|('simce8b2007' in file):                       
        # ajustar escala 2007 y 2008
        data[question].replace([0,1,99],np.nan,inplace=True)
        data[question] = data[question] -1 
    
    if "2002" in file:                       
        data[question] = data.apply(correct_dad_ed_2002, axis=1)
        
    if 'simce8b2004' in file:                       
        data[question] = data.apply(correct_dad_ed_2004, axis=1)
        
    if ("2003" in file):                       
        data[question].replace(dict_mom_ed_two_digit,inplace=True)
     
    if "simce3m2012" in file:
        data.drop_duplicates(keep="first", inplace=True)
       
    if ('simce4b2005' in file):
        data['dad_ed'] = data.cpad_p07+1
        q_list = q_list[:-1] + ['dad_ed']
    
    
    data[q_list[-1]] = data[q_list[-1]].replace([0,98,99,100],np.nan)

    return data, q_list

In [ ]:
uf_pesos = uf_pesos_dict[int(agno)]   
for i in range(1,16):
    data_case.income.replace(i, uf_pesos[i-1],inplace=True)

In [45]:
respondant = pd.read_excel('quien responde y su edad cpad all years.xlsx')

dict_estandar_respondant = {'4b2002':{4:3,5:3,6:3,7:3,8:3,9:3,0:np.nan, 99:np.nan},
                        '2m2003':{4:3,5:3,6:3,7:3,8:3,9:3,0:np.nan, 99:np.nan},
                        '8b2004':{4:3,5:3,6:3,7:3,8:3,9:3,0:np.nan, 99:np.nan},
                        '4b2005':{1:3,0:1,4:3,5:3,6:3,7:3,8:3,99:np.nan},
                        '2m2006':{2:3,3:2,4:3,5:3,6:3,7:3,8:3,9:3,99:np.nan,0:np.nan},
                        '4b2006':{2:3,3:2,4:3,5:3,6:3,7:3,8:3,9:3,99:np.nan,0:np.nan},
                        '4b2007':{2:3,3:2,4:3,5:3,6:3,7:3,8:3,9:np.nan,99:np.nan,0:np.nan},
                        '8b2007':{2:3,3:2,4:3,5:3,6:3,7:3,8:3,9:np.nan,99:np.nan,0:np.nan},
                        '2m2008':{2:3,3:2,4:3,5:3,6:3,7:3,8:3,9:3,99:np.nan,0:np.nan},
                        '4b2008':{2:3,3:2,4:3,5:3,6:3,7:3,8:3,9:3,99:np.nan,0:np.nan},
                        '4b2009':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2009':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2010':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2010':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2011':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2011':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2b2012':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2012':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '3m2012':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2012':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2b2013':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2013':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2013':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '6b2013':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2013':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2b2014':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2014':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2014':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '6b2014':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2014':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2b2015':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2015':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2015':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '6b2015':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2015':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2016':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2016':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '6b2016':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2017':{1:12,2:3,4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2017':{1:12,2:3,4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2017':{1:12,2:3,4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2018':{1:12,2:3,4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2018':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '6b2018':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '8b2019':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '4b2022':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan},
                        '2m2022':{4:3,5:3,6:3,7:3,8:3,9:3, 0:np.nan, 99:np.nan}}

In [46]:
gy_list = respondant.grade_year.unique()
dict_relacion = defaultdict(lambda: np.nan)
dict_relacion_age = defaultdict(lambda: np.nan)
for gy in gy_list: 
    data = respondant[respondant.grade_year == gy]
    for i in [0,1]:
        try:
            if 'relación' in data.iloc[i]['Descripción']:
                dict_relacion[gy] = data.iloc[i]['Nombre']
            else:
                dict_relacion_age[gy] = data.iloc[i]['Nombre']
        except:
            pass

In [47]:
def add_responsent_age(data,file, dict_relacion_age, q_list, dict_estandar_respondant):

    q_respond = dict_relacion[curso + agno]
    q_age = dict_relacion_age[curso + agno]
    
    print(q_age,q_respond)

    if str(q_age)!= 'nan':
        data.rename(columns = {q_respond:'respondant', q_age:'respondant_age'},inplace=True)
    else:
        data['q_age'] = np.nan
        data.rename(columns = {q_respond:'respondant', 'q_age':'respondant_age'},inplace=True)

    print(curso + agno)
    data.respondant_age.replace({0:np.nan, 999:np.nan, 474:np.nan},inplace=True)
    data.respondant.replace(dict_estandar_respondant[curso+agno],inplace=True)
    data.respondant.replace({'P':2},inplace=True)

    return data, q_list + ['respondant','respondant_age']

In [48]:
perm_cols = ['agno','curso','idalumno','rbd']
panel_parents = pd.DataFrame(columns=['agno','idalumno','rbd','parental_exp','income','mom_ed','dad_ed','respondant','respondant_age'])
for traj in [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11_pandemia,T12,T13,T14,T15,CS1,CS2,CS3,CS4, CS5_pandemia]: 
    print(traj)
    for t in traj:
        curso =  t[5:7]
        agno = t[:4]
        for root, dirs, files in os.walk(path):
            # walk all files in subfolders
             for file in files:
                # work with cpad 
                if curso+agno+'_cpad' in file and (('csv' in file) | ('txt' in file)):
                    
                    if ('simce2m2014' in file):
                        data = pd.read_excel(path + r'\2014\Simce2m2014_privadas-SEG\Archivos XLS (XLSX)\simce2m2014_cpad_privada_final-SEG.xlsx')
                    else: 
                        path_file = os.path.join(root, file)
                        data = pd.read_csv(path_file,encoding = 'unicode_escape', sep="|")
 
                    data['agno'] = int(agno)
                    data['curso'] = curso
                    data.columns = [x.lower() for x in data.columns ]

                    print(file)
                
                    data, q_list = add_parental_expectations(data,file, parents_match_dict)
                    data, q_list = add_income(data,file, parents_match_dict, q_list)
                    data, q_list = add_mom_ed(data,file, parents_match_dict, q_list)
                    data, q_list = add_dad_ed(data,file, parents_match_dict, q_list)
                    data, q_list = add_responsent_age(data,file, dict_relacion_age, q_list, dict_estandar_respondant)
                                    
                    data_case = data[perm_cols + q_list]
                    data_case.columns = ['agno','curso','idalumno','rbd','parental_exp', 'income', 'mom_ed', 'dad_ed','respondant','respondant_age']
                    data_case.parental_exp.replace(9, np.nan, inplace=True)
                    panel_parents = pd.concat([panel_parents,data_case])
                    
panel_parents.dad_ed.replace(69,np.nan,inplace=True)
panel_parents.parental_exp.replace(99, np.nan, inplace=True)

['2005_4b_cpad', '2009_8b_cpad', '2012_3m_cpad']
simce4b2005_cpad_privada_final_SEG.csv
nan cpad_p01
4b2005
simce8b2009_cpad_privadas_final-SEG.csv
cpad_p02 cpad_p03
8b2009
simce3m2012_cpad_publica_final-SEG.csv
cpad_p02 cpad_p03
3m2012
['2007_4b_cpad', '2011_8b_cpad', '2013_2m_cpad']
simce4b2007_cpad_privada_final-SEG.csv
nan cpad_p01
4b2007
simce8b2011_cpad_privada_final-SEG.csv
cpad_p02 cpad_p03
8b2011
simce2m2013_cpad_privada_final-SEG.csv
cpad_p02 cpad_p03
2m2013
['2008_4b_cpad', '2014_2m_cpad']
simce4b2008_cpad_privada_final-SEG.csv
nan cpad_p01
4b2008
simce2m2014_cpad_privada_final-SEG.csv
cpad_p03 cpad_p01
2m2014
['2009_4b_cpad', '2013_8b_cpad', '2015_2m_cpad']
simce4b2009_cpad_privadas_final-SEG.csv
cpad_p02 cpad_p03
4b2009
simce8b2013_cpad_privada_final-SEG.csv
cpad_p02 cpad_p03
8b2013
simce2m2015_cpad_privadas_final-SEG.csv
cpad_p02 cpad_p03
2m2015
['2010_4b_cpad', '2014_8b_cpad', '2016_2m_cpad']
simce4b2010_cpad_privadas_final-SEG.csv
cpad_p02 cpad_p03
4b2010
simce8b2014_cp

In [330]:
panel_parents['parental_exp'] = panel_parents.parental_exp.replace(0, np.nan)

In [331]:
panel_parents.to_csv('Datos\Intermedios\Agencia\20231027_panel_cuestionario_padres.csv', index=False)

## Teachers

In [306]:
match_df_age_question_t.loc[match_df_age_question_t.grade_year == '4b2015_cprof_prompt', 'grade_year'] = '4b2015_cprof_lect'
match_df_age_question_t.loc[match_df_age_question_t.grade_year == '4b2022_cprof_prompt', 'key'] = 'cprof_p02_01'

match_df_class_code_question.loc[match_df_class_code_question.grade_year == '2m2018_cprof_mate', 'key'] = 'codigoCurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '8b2019_cprof_lect', 'key'] = 'codigocurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '8b2019_cprof_mate', 'key'] = 'codigocurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '2m2022_cprof_lect', 'key'] = 'codigoCurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '2m2022_cprof_mate', 'key'] = 'codigoCurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '4b2022_cprof_prompt', 'key'] = 'codigoCurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '4b2005_cprof_prompt', 'key'] = 'codigocurso'
match_df_class_code_question.loc[match_df_class_code_question.grade_year == '4b2002_cprof_lect', 'key'] = 'letra_curso'

teachers_match_dict = {
    'teacher_age':match_df_age_question_t,
    'grade_level':match_df_grade_level_question,
    'class_code':match_df_class_code_question,
    'class_letter': match_df_class_letter_question,
    'teacher_expectations': match_df_teacher_expectations
}

In [307]:
perm_cols = ['agno','curso', 'rbd']
panel_teacher = pd.DataFrame(columns=['agno','curso','cod_curso','rbd'])
for traj in [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11_pandemia,T12,T13,T14,T15,CS1,CS2,CS3,CS4, CS5_pandemia]: 
    for t in traj:
        curso =  t[5:7]
        agno = t[:4]
        for root, dirs, files in os.walk(path):
            # walk all files in subfolders
             for file in files:
                # work with cpad 
                if curso+agno+'_cprof' in file and (('csv' in file) | ('txt' in file)):
                    path_file = os.path.join(root, file)
                    if '6b2018' in file:
                        data = pd.read_csv(path_file,encoding = 'unicode_escape', sep=",")
                    else:
                        data = pd.read_csv(path_file,encoding = 'unicode_escape', sep="|")
                    
                    data['agno'] = int(agno)
                    data['curso'] = curso
                    
                    if ('_lect' in file):                    
                        subject = 'reading'
                        suf = '_lect'
                    elif ('_mat' in file):
                        subject = 'math'
                        suf = '_mat'
                    elif ('_nat' in file):
                        subject = 'cience'
                        suf = '_nat'
                    elif ('_soc' in file):
                        subject = 'history'
                        suf = '_soc'
                    else:
                        subject = None
                        suf = ''
                        
                    data['subject'] = subject
                    
                    print(file)
                    q_list = []
                    for q in ['teacher_expectations']: #, 'teacher_age', 'class_code', 'teacher_expectations']:
                        match_df = teachers_match_dict[q]
                        
                        case = file.split('_')[0][5:] + '_' + file.split('_')[1] + suf
                        question = match_df[match_df.grade_year.str.contains(case)].key.values[0]
                        print(question)
                        
                        if ('fecha' not in str(question)) & (str(question)!= 'nan'):
                            q_list = q_list + [question]
                            
                            print(data[question].value_counts(dropna=False).sort_index()/len(data))
                                                
                    data_case = data[perm_cols + q_list]
                    ##data_case.replace(0,np.nan,inplace=True)
                    ##data_case.replace(99,np.nan,inplace=True)
                    #data_case.columns = ['agno','curso','rbd','teacher_age']
                    ##
                    #panel_teacher = pd.concat([panel_teacher,data_case])

simce4b2005_cprof_privada_final_SEG.csv
cprof_p16
cprof_p16
0.0     0.000669
1.0     0.015570
2.0     0.014424
3.0     0.160856
4.0     0.058363
5.0     0.096857
6.0     0.121884
7.0     0.007928
99.0    0.016907
NaN     0.506543
Name: count, dtype: float64
simce8b2009_cprof_lect_privada_final_SEG.csv
cprof_p22
cprof_p22
0.0     0.049313
1.0     0.024330
2.0     0.136167
3.0     0.060301
4.0     0.129235
5.0     0.138260
6.0     0.007325
99.0    0.004186
NaN     0.450883
Name: count, dtype: float64
simce8b2009_cprof_mate_privada_final_SEG.csv
cprof_p22
cprof_p22
0.0     0.050615
1.0     0.026681
2.0     0.139942
3.0     0.067225
4.0     0.117709
5.0     0.134319
6.0     0.006801
99.0    0.003270
NaN     0.453440
Name: count, dtype: float64
simce8b2009_cprof_nat_privada_final_SEG.csv
cprof_p22
cprof_p22
0.0     0.048095
1.0     0.023522
2.0     0.132983
3.0     0.067937
4.0     0.122076
5.0     0.138371
6.0     0.008016
99.0    0.005256
NaN     0.453745
Name: count, dtype: float64
simce

simce6b2015_cprof_lect_privada_final_SEG.csv
nan
simce6b2015_cprof_mate_privada_final_SEG.csv
nan
simce8b2017_cprof_mate_privada_final_SEG.csv
cprof_p23
cprof_p23
0.0     0.009370
1.0     0.010452
2.0     0.072681
3.0     0.031595
4.0     0.136112
5.0     0.230899
6.0     0.024027
99.0    0.006127
NaN     0.478736
Name: count, dtype: float64
simce8b2017_cprof_nat_privada_final_SEG.csv
cprof_p23
cprof_p23
0.0     0.012094
1.0     0.010994
2.0     0.080625
3.0     0.036404
4.0     0.129001
5.0     0.225018
6.0     0.019790
99.0    0.007085
NaN     0.478989
Name: count, dtype: float64
simce4b2014_cprof_privada_final_SEG.csv
cprof_p06
cprof_p06
0.0     0.003789
1.0     0.002867
2.0     0.047921
3.0     0.012083
4.0     0.100041
5.0     0.218308
6.0     0.032767
99.0    0.005632
NaN     0.576592
Name: count, dtype: float64
simce6b2016_cprof_lect_privada_final_SEG.csv
cprof_p10
cprof_p10
0.0     0.005101
1.0     0.008502
2.0     0.068544
3.0     0.030393
4.0     0.099362
5.0     0.186504
6.0

(99, 9, 0)

## Scores

In [320]:
perm_cols = ['agno','curso','cod_curso','idalumno', 'rbd']
panel_alu = pd.DataFrame(columns=['agno','curso','cod_curso','idalumno','rbd','simce_mat','simce_lect'])
for traj in [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11_pandemia,T12,T13,T14,T15,CS1,CS2,CS3,CS4, CS5_pandemia]: 
    for t in traj:
        curso =  t[5:7]
        agno = t[:4]
        for root, dirs, files in os.walk(path):
            # walk all files in subfolders
             for file in files:
                # work with cpad 
                if curso+agno+'_alu' in file and (('csv' in file) | ('txt' in file)):
                    path_file = os.path.join(root, file)
                    data = pd.read_csv(path_file,encoding = 'unicode_escape', sep="|")
                    data['agno'] = int(agno)
                    data['curso'] = curso
                    print(file)
                    q_list = []
                    
                    for q in ['math','reading']:
                        match_df = alu_match_dict[q]
                        question = match_df[match_df.grade_year == file[5:15]].key.values[0]                   
                        q_list = q_list + [question]
                                                
                    data_case = data[perm_cols + q_list]
                    data_case.replace(0,np.nan,inplace=True)
                    data_case.replace(99,np.nan,inplace=True)
                    data_case.columns = ['agno','curso','cod_curso','idalumno','rbd','simce_mat','simce_lect']
                    
                    panel_alu = pd.concat([panel_alu,data_case])

simce4b2005_alu_mrun_privada_final-SEG.csv
simce8b2009_alu_mrun_privadas_final-SEG.csv
simce3m2012_alu_mrun_publica_final-SEG.csv
simce4b2007_alu_mrun_privada_final-SEG.csv
simce8b2011_alu_mrun_privada_final-SEG.csv
simce2m2013_alu_mrun_privada_final-SEG.csv
simce4b2008_alu_mrun_privada_final-SEG.csv
simce2m2014_alu_mrun_privada_final-SEG.csv
simce4b2009_alu_mrun_privadas_final-SEG.csv
simce8b2013_alu_mrun_privada_final-SEG.csv
simce2m2015_alu_mrun_privadas_final-SEG.csv
simce4b2010_alu_mrun_privadas_final-SEG.csv
simce8b2014_alu_mrun_privada_final-SEG.csv
simce2m2016_alu_mrun_privada_final_SEG.csv
simce4b2011_alu_mrun_privada_final-SEG.csv
simce6b2013_alu_mrun_privada_final-SEG.csv
simce8b2015_alu_mrun_privadas_final-SEG.csv
simce2m2017_alu_mrun_privada_final_SEG.csv
simce4b2012_alu_mrun_privada_final-SEG.csv
simce6b2014_alu_mrun_privada_final-SEG.csv
simce2m2018_alu_privada_final-SEG.txt
simce4b2013_alu_mrun_privada_final-SEG.csv
simce6b2015_alu_mrun_privadas_final-SEG.csv
simce8b201

In [333]:
panel_alu.to_csv('Datos\Intermedios\Agencia\20231027_panel_puntaje_estudiantes.csv',index=False)

## Students

Pensando en tu futuro, ¿cuál crees tú que es el nivel de educación más alto que vas a completar?

1: No creo completar 4° año Ed. Media
2: 4° Año Ed. Media
3: Una carrera Instituto Prof o Centro form. Tec.
4: Una carrera en una Universidad.
0: Vacío
99: Doble marca
    
    
1: No creo que llegue a completar IV año de educación media
2: Completaré IV año de educación media científico humanista (CH)
3: Completaré IV año de educación media técnico profesional (TP)
4: Completaré una carrera en un centro de formación técnica o instituto profesional
5: Completaré una carrera en una universidad
6: Completaré un programa de posgrado (magíster o doctorado)
0: Vacío
99: Doble marca

In [126]:
# lo anterior par aaspiaciones de estudiantes 
panel_asp_alu = pd.DataFrame(columns=['agno','curso','idalumno', 'rbd','student_expectations'])
for traj in [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11_pandemia,T12,T13,T14,T15,CS1,CS2,CS3,CS4, CS5_pandemia]: 
    for t in traj:
        curso =  t[5:7]
        agno = t[:4]
        if (curso in ['8b', '2m']) & (int(agno) >= 2009):
            for root, dirs, files in os.walk(path):
                # walk all files in subfolders
                 for file in files:
                    # work with cest
                    if (curso+agno+'_cest' in file and (('csv' in file) | ('txt' in file))) & (curso+agno+'_cest' in est_asp_real):
                        path_file = os.path.join(root, file)
                        data = pd.read_csv(path_file,encoding = 'unicode_escape', sep="|")
                        data['agno'] = int(agno)
                        data['curso'] = curso
                        print(file)
                        
                        question = student_expectations[student_expectations.grade_year == file[5:12] + 'cest'].key.values[0]
                        
                        if '2m2016_cestfb' in file:
                            question = 'cest_p08'
                        
                        print(question)
                        
                        data_case = data[['agno','curso','idalumno', 'rbd'] + [question]]
                        data_case.replace(9,np.nan,inplace=True)
                        data_case.replace(0,np.nan,inplace=True)
                        data_case.replace(99,np.nan,inplace=True)
                        
                        data_case['student_exp_coordinada'] = data_case[question]
                       
                        if int(agno) >=2017:
                            data_case.student_exp_coordinada.replace(3,2,inplace=True)
                            data_case.student_exp_coordinada.replace(4,3,inplace=True)
                            data_case.student_exp_coordinada.replace(5,4,inplace=True)
                            data_case.student_exp_coordinada.replace(6,4,inplace=True)

                        
                        data_case.columns = ['agno','curso','idalumno', 'rbd','student_expectations','student_exp_coordinada']                        
                        panel_asp_alu = pd.concat([panel_asp_alu,data_case])


simce8b2009_cest_privadas_final-SEG.csv
cest_p23
simce8b2011_cest_privada_final-SEG.csv
cest_p05
simce2m2013_cest_privada_final-SEG.csv
cest_p06
simce2m2014_cest_privada_final-SEG.csv
cest_p07
simce8b2013_cest_privada_final-SEG.csv
cest_p06
simce2m2015_cest_privadas_final-SEG.csv
cest_p06
simce8b2014_cest_privada_final-SEG.csv
cest_p07
simce2m2016_cestfa_privada_final_SEG.csv
cest_p11
simce2m2016_cestfb_privada_final_SEG.csv
cest_p08
simce8b2015_cest_privadas_final-SEG.csv
cest_p06
simce2m2017_cestfa_privada_final_SEG.csv
cest_p41
simce2m2017_cestfb_privada_final_SEG.csv
cest_p41
simce2m2018_cest_privada_final-SEG.txt
cest_p04
simce8b2017_cestfa_final_SEG.csv
cest_p41
simce8b2017_cestfb_final_SEG.csv
cest_p41
simce8b2019_cest_final_SEG.csv
cest_p06
simce2m2012_cest_privada_final-SEG.csv
cest_p09
simce2m2010_cest_privadas_final-SEG.csv
cest_p23


In [332]:
panel_asp_alu.to_csv('Datos\Intermedios\Agencia\20231027_panel_cuestionario_estudiantes.csv', index=False)